# L5: Automate Recipie Finder

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [14]:
from crewai import Agent, Crew, Task

In [22]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [36]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [37]:
recipe_finder = Agent(
    role='AI Recipe Finder',
    goal=(
        'Find the best recipes based on the given ingredients. Ensure that the recipe does not contain more than '
        '1 or 2 extra ingredients compared to the provided list. The goal is to suggest recipes with minimal additional ingredients.'
    ),
    backstory=(
        "An expert AI chef who finds the best recipes using online resources. The AI should prioritize recipes with "
        "only 1 or 2 extra ingredients to avoid overwhelming the provided ingredients."
    ),
    tools=[search_tool],  # Attach SerperDevTool for web search to find recipes
    verbose=True,
    memory=True
)


In [38]:
ingredient_checker = Agent(
    role='Quality Assurance Head',
    goal='Identify extra ingredients used in a recipe that were not provided in the ingredients list.',
    backstory=(
        "You are an expert AI chef specializing in quality assurance for recipes. "
        "Your job is to compare the ingredients used in a recipe against the provided ingredient list, "
        "identify any extra ingredients, and check their availability and price on Indian grocery store apps."
    ),
    tools=[search_tool],  # Attach SerperDevTool for web search
    verbose=True
)

In [39]:
recipe_finder_task = Task(
    description=(
        "Perform a web search to find the best possible recipe using the given ingredients: {ingredients}. "
        "Extract the most relevant and easy-to-follow recipe and provide:\n\n"
        "- The recipe name\n"
        "- A short description\n"
        "- Step-by-step cooking instructions\n"
        "- Estimated preparation & cooking time\n"
        "- Any additional useful details\n\n"
        "Make sure the recipe is simple and practical for everyday cooking."
    ),
    expected_output="A well-structured recipe including name, description, steps, and cooking time.",
    tools=[search_tool],  # Attach SerperDevTool to perform search queries
    agent=recipe_finder
)

In [40]:
ingredient_check_task = Task(
    description=(
        "You are given the following **provided ingredients**: {ingredients}.\n"
        "You are also given a **recipe** that contains the following ingredients: {ingredients}.\n\n"
        "**Your task:**\n"
        "1. Compare the provided ingredients with the recipe to identify any extra ingredients not in the provided list.\n"
        "2. Provide a detailed explanation of the recipe, including key steps, ingredients, and any notable techniques.\n"
        "3. List any extra ingredients found in the recipe that are not present in the provided ingredient list.\n\n"
        "**Your final output MUST include:**\n"
        "- Explanation of the recipe and its key steps.\n"
        "- List of extra ingredients found in the recipe.\n"
        "- A short summary of whether the extra ingredients significantly impact the dish."
    ),
    expected_output=(
        "A structured report listing:\n"
        "- Explanation of the recipe\n"
        "- List of extra ingredients used in the recipe\n"
        "- Short summary on the impact of extra ingredients on the dish"
    ),
    tools=[search_tool],  # Attach SerperDevTool for any optional price checks or additional tools if needed in the future
    agent=ingredient_checker
)


In [41]:
crew = Crew(
    agents=[recipe_finder,ingredient_checker],
    tasks=[recipe_finder_task,ingredient_check_task]
)

# Run the Crew with Sample Input
result = crew.kickoff(inputs={'ingredients': 'potato, 2 chili,onion'})
print(result)

2025-02-03 18:47:57,447 - 140343251458944 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
I need to search the internet to find a recipe that includes potatoes, chili, and onion with minimal additional ingredients.

Action: Search the internet
Action Input: {"search_query": "potato chili onion recipe minimal ingredients"} 


Search results: Title: Chili Recipe - Allrecipes
Link: https://www.allrecipes.com/recipe/15386/chili-iii/
Snippet: Ingredients ; 1 pound ground beef ; 1 onion, chopped ; 4 potatoes, cubed ; 2 cups water, or as needed ; 2 cups tomato puree.
---
Title: Chili with Potatoes - Recipe for Perfection
Link: https://recipeforperfection.com/chili-with-potatoes/
Snippet: Add the beans, tomatoes, chili powder, sugar, pepper, oregano, and cooked ground beef to a large pot. Stir, then bring to a gentle boil over ...
---
Title: Garlic Chili Potatoes - Piping Pot Curry
Link: https://pipingpotcurry.com/garlic-chili-potatoes/
Snippet: Flavorful and easy Garlic Chili Potatoes, made with steamed potatoes, garlic, chili sauce and 